# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [30]:
import requests
import json
import pandas as pd

url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"


response = requests.get(url)

if response.status_code == 200:
        print("All good!")
        print("==============")
        print("\n")




All good!




In [2]:
response.content

b'{"nobelPrizes":[{"awardYear":"1901","category":{"en":"Chemistry","no":"Kjemi","se":"Kemi"},"categoryFullName":{"en":"The Nobel Prize in Chemistry","no":"Nobelprisen i kjemi","se":"Nobelpriset i kemi"},"dateAwarded":"1901-11-12","prizeAmount":150782,"prizeAmountAdjusted":10531894,"links":[{"rel":"nobelPrize","href":"https://api.nobelprize.org/2/nobelPrize/che/1901","action":"GET","types":"application/json"}],"laureates":[{"id":"160","knownName":{"en":"Jacobus H. van \'t Hoff"},"fullName":{"en":"Jacobus Henricus van \'t Hoff"},"portion":"1","sortOrder":"1","motivation":{"en":"in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions","se":"s\xc3\xa5som ett erk\xc3\xa4nnande av den utomordentliga f\xc3\xb6rtj\xc3\xa4nst han inlagt genom uppt\xc3\xa4ckten av lagarna f\xc3\xb6r den kemiska dynamiken och f\xc3\xb6r det osmotiska trycket i l\xc3\xb6sningar"},"links":[{"rel":"laureate","href":"https://api.

In [3]:
response.headers

{'Date': 'Sun, 25 Aug 2024 16:12:32 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Headers': 'X-Requested-With, Content-Type, Accept, Origin, Authorization', 'Access-Control-Allow-Methods': 'GET', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'public, max-age=60', 'Expires': 'Sun, 25 Aug 2024 16:13:32 GMT', 'Pragma': 'no-cache', 'X-Powered-By': 'PHP/8.2.20', 'CF-Cache-Status': 'HIT', 'Age': '329', 'Last-Modified': 'Sun, 25 Aug 2024 16:07:03 GMT', 'Vary': 'Accept-Encoding', 'Server': 'cloudflare', 'CF-RAY': '8b8cdd3f19ea194b-FRA', 'Content-Encoding': 'br', 'alt-svc': 'h3=":443"; ma=86400'}

In [4]:
info = response.json ()
info

{'nobelPrizes': [{'awardYear': '1901',
   'category': {'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'},
   'categoryFullName': {'en': 'The Nobel Prize in Chemistry',
    'no': 'Nobelprisen i kjemi',
    'se': 'Nobelpriset i kemi'},
   'dateAwarded': '1901-11-12',
   'prizeAmount': 150782,
   'prizeAmountAdjusted': 10531894,
   'links': [{'rel': 'nobelPrize',
     'href': 'https://api.nobelprize.org/2/nobelPrize/che/1901',
     'action': 'GET',
     'types': 'application/json'}],
   'laureates': [{'id': '160',
     'knownName': {'en': "Jacobus H. van 't Hoff"},
     'fullName': {'en': "Jacobus Henricus van 't Hoff"},
     'portion': '1',
     'sortOrder': '1',
     'motivation': {'en': 'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions',
      'se': 'såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av lagarna för den kemiska dynamiken och för det osmotiska tryck

In [5]:
type (info)

dict

In [6]:
info.keys()

dict_keys(['nobelPrizes', 'meta', 'links'])

In [7]:
info ['nobelPrizes']

[{'awardYear': '1901',
  'category': {'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'},
  'categoryFullName': {'en': 'The Nobel Prize in Chemistry',
   'no': 'Nobelprisen i kjemi',
   'se': 'Nobelpriset i kemi'},
  'dateAwarded': '1901-11-12',
  'prizeAmount': 150782,
  'prizeAmountAdjusted': 10531894,
  'links': [{'rel': 'nobelPrize',
    'href': 'https://api.nobelprize.org/2/nobelPrize/che/1901',
    'action': 'GET',
    'types': 'application/json'}],
  'laureates': [{'id': '160',
    'knownName': {'en': "Jacobus H. van 't Hoff"},
    'fullName': {'en': "Jacobus Henricus van 't Hoff"},
    'portion': '1',
    'sortOrder': '1',
    'motivation': {'en': 'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions',
     'se': 'såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av lagarna för den kemiska dynamiken och för det osmotiska trycket i lösningar'},
    'links': [{'

In [34]:
pd.set_option('display.max_columns', None) 
df

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN
4,1901,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",{'en': 'The Nobel Prize in Physiology or Medic...,1901-10-30,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN
...,...,...,...,...,...,...,...,...,...
665,2023,"{'en': 'Economic Sciences', 'no': 'Økonomi', '...",{'en': 'The Sveriges Riksbank Prize in Economi...,2023-10-09,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1034', 'knownName': {'en': 'Claudia G...",NaN
666,2023,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",2023-10-05,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1032', 'knownName': {'en': 'Jon Fosse...",NaN
667,2023,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",2023-10-06,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1033', 'knownName': {'en': 'Narges Mo...",NaN
668,2023,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",2023-10-03,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1026', 'knownName': {'en': 'Pierre Ag...",NaN


In [8]:
info ['meta']

{'offset': 0,
 'limit': 100000,
 'count': 670,
 'terms': 'https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/',
 'license': 'https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#licence',
 'disclaimer': 'https://www.nobelprize.org/about/terms-of-use-for-api-nobelprize-org-and-data-nobelprize-org/#disclaimer'}

In [9]:
info ['links']

{'first': 'http://nobel-external-api-app.azurewebsites.net/2.1/nobelPrizes?offset=0&limit=100000',
 'self': 'http://nobel-external-api-app.azurewebsites.net/2.1/nobelPrizes?offset=0&limit=100000',
 'last': 'http://nobel-external-api-app.azurewebsites.net/2.1/nobelPrizes?offset=0&limit=100000'}

In [38]:
df = pd.DataFrame(response)
df.head()

,0
0,"b'{""nobelPrizes"":[{""awardYear"":""1901"",""categor..."
1,"b'rize in Chemistry"",""no"":""Nobelprisen i kjemi..."
2,"b'mountAdjusted"":10531894,""links"":[{""rel"":""nob..."
3,"b'ypes"":""application/json""}],""laureates"":[{""id..."
4,"b'us van \'t Hoff""},""portion"":""1"",""sortOrder"":..."


# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmountAdjusted
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [11]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes']

df = pd.DataFrame(prizes_list)
df

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,links,laureates,topMotivation
0,1901,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}","{'en': 'The Nobel Prize in Chemistry', 'no': '...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN
1,1901,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",1901-11-14,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN
2,1901,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",1901-12-10,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN
3,1901,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",1901-11-12,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN
4,1901,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",{'en': 'The Nobel Prize in Physiology or Medic...,1901-10-30,150782,10531894,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN
...,...,...,...,...,...,...,...,...,...
665,2023,"{'en': 'Economic Sciences', 'no': 'Økonomi', '...",{'en': 'The Sveriges Riksbank Prize in Economi...,2023-10-09,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1034', 'knownName': {'en': 'Claudia G...",NaN
666,2023,"{'en': 'Literature', 'no': 'Litteratur', 'se':...","{'en': 'The Nobel Prize in Literature', 'no': ...",2023-10-05,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1032', 'knownName': {'en': 'Jon Fosse...",NaN
667,2023,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}","{'en': 'The Nobel Peace Prize', 'no': 'Nobels ...",2023-10-06,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1033', 'knownName': {'en': 'Narges Mo...",NaN
668,2023,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}","{'en': 'The Nobel Prize in Physics', 'no': 'No...",2023-10-03,11000000,11000000,"[{'rel': 'nobelPrize', 'href': 'https://api.no...","[{'id': '1026', 'knownName': {'en': 'Pierre Ag...",NaN


In [12]:
df_filtered = df[['category', 'dateAwarded', 'prizeAmount', 'prizeAmountAdjusted', 'laureates', 'topMotivation']]
display(df_filtered)

,category,dateAwarded,prizeAmount,prizeAmountAdjusted,laureates,topMotivation
0,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}",1901-11-12,150782,10531894,"[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN
1,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",1901-11-14,150782,10531894,"[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN
2,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",1901-12-10,150782,10531894,"[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN
3,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",1901-11-12,150782,10531894,"[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN
4,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",1901-10-30,150782,10531894,"[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN
...,...,...,...,...,...,...
665,"{'en': 'Economic Sciences', 'no': 'Økonomi', '...",2023-10-09,11000000,11000000,"[{'id': '1034', 'knownName': {'en': 'Claudia G...",NaN
666,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",2023-10-05,11000000,11000000,"[{'id': '1032', 'knownName': {'en': 'Jon Fosse...",NaN
667,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",2023-10-06,11000000,11000000,"[{'id': '1033', 'knownName': {'en': 'Narges Mo...",NaN
668,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",2023-10-03,11000000,11000000,"[{'id': '1026', 'knownName': {'en': 'Pierre Ag...",NaN


In [13]:
df_filtered.columns = df_filtered.columns.str.lower()
df_filtered

,category,dateawarded,prizeamount,prizeamountadjusted,laureates,topmotivation
0,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}",1901-11-12,150782,10531894,"[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN
1,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",1901-11-14,150782,10531894,"[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN
2,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",1901-12-10,150782,10531894,"[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN
3,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",1901-11-12,150782,10531894,"[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN
4,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",1901-10-30,150782,10531894,"[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN
...,...,...,...,...,...,...
665,"{'en': 'Economic Sciences', 'no': 'Økonomi', '...",2023-10-09,11000000,11000000,"[{'id': '1034', 'knownName': {'en': 'Claudia G...",NaN
666,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",2023-10-05,11000000,11000000,"[{'id': '1032', 'knownName': {'en': 'Jon Fosse...",NaN
667,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",2023-10-06,11000000,11000000,"[{'id': '1033', 'knownName': {'en': 'Narges Mo...",NaN
668,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",2023-10-03,11000000,11000000,"[{'id': '1026', 'knownName': {'en': 'Pierre Ag...",NaN


In [14]:
df_filtered['laureates'].apply(type).head()

0    <class 'list'>
1    <class 'list'>
2    <class 'list'>
3    <class 'list'>
4    <class 'list'>
Name: laureates, dtype: object

In [15]:
df_filtered['laureates'].iloc[0]

[{'id': '160',
  'knownName': {'en': "Jacobus H. van 't Hoff"},
  'fullName': {'en': "Jacobus Henricus van 't Hoff"},
  'portion': '1',
  'sortOrder': '1',
  'motivation': {'en': 'in recognition of the extraordinary services he has rendered by the discovery of the laws of chemical dynamics and osmotic pressure in solutions',
   'se': 'såsom ett erkännande av den utomordentliga förtjänst han inlagt genom upptäckten av lagarna för den kemiska dynamiken och för det osmotiska trycket i lösningar'},
  'links': [{'rel': 'laureate',
    'href': 'https://api.nobelprize.org/2/laureate/160',
    'action': 'GET',
    'types': 'application/json'}]}]

In [16]:
df_filtered.loc[:,'laureate_ids'] = df['laureates'].apply(lambda x: [laureate['id'] for laureate in x] if isinstance(x, list) else [])
df_filtered

C:\Users\Nicole\AppData\Local\Temp\ipykernel_10248\3881116552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:,'laureate_ids'] = df['laureates'].apply(lambda x: [laureate['id'] for laureate in x] if isinstance(x, list) else [])


,category,dateawarded,prizeamount,prizeamountadjusted,laureates,topmotivation,laureate_ids
0,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}",1901-11-12,150782,10531894,"[{'id': '160', 'knownName': {'en': 'Jacobus H....",NaN,[160]
1,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",1901-11-14,150782,10531894,"[{'id': '569', 'knownName': {'en': 'Sully Prud...",NaN,[569]
2,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",1901-12-10,150782,10531894,"[{'id': '462', 'knownName': {'en': 'Henry Duna...",NaN,"[462, 463]"
3,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",1901-11-12,150782,10531894,"[{'id': '1', 'knownName': {'en': 'Wilhelm Conr...",NaN,[1]
4,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",1901-10-30,150782,10531894,"[{'id': '293', 'knownName': {'en': 'Emil von B...",NaN,[293]
...,...,...,...,...,...,...,...
665,"{'en': 'Economic Sciences', 'no': 'Økonomi', '...",2023-10-09,11000000,11000000,"[{'id': '1034', 'knownName': {'en': 'Claudia G...",NaN,[1034]
666,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",2023-10-05,11000000,11000000,"[{'id': '1032', 'knownName': {'en': 'Jon Fosse...",NaN,[1032]
667,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",2023-10-06,11000000,11000000,"[{'id': '1033', 'knownName': {'en': 'Narges Mo...",NaN,[1033]
668,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",2023-10-03,11000000,11000000,"[{'id': '1026', 'knownName': {'en': 'Pierre Ag...",NaN,"[1026, 1027, 1028]"


In [17]:
df_nobels = df_filtered[['category', 'dateawarded', 'prizeamount', 'prizeamountadjusted', 'laureate_ids', 'topmotivation']]
df_nobels

,category,dateawarded,prizeamount,prizeamountadjusted,laureate_ids,topmotivation
0,"{'en': 'Chemistry', 'no': 'Kjemi', 'se': 'Kemi'}",1901-11-12,150782,10531894,[160],NaN
1,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",1901-11-14,150782,10531894,[569],NaN
2,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",1901-12-10,150782,10531894,"[462, 463]",NaN
3,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",1901-11-12,150782,10531894,[1],NaN
4,"{'en': 'Physiology or Medicine', 'no': 'Fysiol...",1901-10-30,150782,10531894,[293],NaN
...,...,...,...,...,...,...
665,"{'en': 'Economic Sciences', 'no': 'Økonomi', '...",2023-10-09,11000000,11000000,[1034],NaN
666,"{'en': 'Literature', 'no': 'Litteratur', 'se':...",2023-10-05,11000000,11000000,[1032],NaN
667,"{'en': 'Peace', 'no': 'Fred', 'se': 'Fred'}",2023-10-06,11000000,11000000,[1033],NaN
668,"{'en': 'Physics', 'no': 'Fysikk', 'se': 'Fysik'}",2023-10-03,11000000,11000000,"[1026, 1027, 1028]",NaN


In [22]:
df_nobels['dateawarded'] = pd.to_datetime(df_nobels['dateawarded'])

df_nobels['dateawarded'] = df_nobels['dateawarded'].dt.strftime('%Y-%m-%d')

C:\Users\Nicole\AppData\Local\Temp\ipykernel_10248\361849379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nobels['dateawarded'] = pd.to_datetime(df_nobels['dateawarded'])
C:\Users\Nicole\AppData\Local\Temp\ipykernel_10248\361849379.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nobels['dateawarded'] = df_nobels['dateawarded'].dt.strftime('%Y-%m-%d')


In [26]:
df_nobels.loc[:,'dateawarded'] = pd.to_datetime(df_nobels['dateawarded'])
df_nobels.loc[:,'dateawarded'] = df_nobels['dateawarded'].dt.strftime('%Y-%m-%d')


AttributeError: Can only use .dt accessor with datetimelike values

# Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [47]:
import time
from tqdm import tqdm


ids = [int(item) for l in df['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):
    if 'fullName' in laureate['knownName']:
        return laureate['knownName']['fullName']['en']
    elif 'orgName' in laureate:
        return laureate['orgName']['en']
    return "Unknown"


def get_gender(laureate):
    if 'gender' in laureate:
        return laureate['gender']
    elif 'orgName' in laureate:
        return "None"
    return "Unknown"

def get_birthdate(laureate):
    if 'birth' in laureate:
        if 'date' in laureate['birth']:
            return laureate['birth']['date']
    return "Unknown"

def get_city(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('en', 'Unknown')
    return "Unknown"

def get_country(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('country', {}).get('en', 'Unknown')
    return "Unknown"

def get_continent(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('continent', {}).get('en', 'Unknown')
    return "Unknown"

def get_latitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('latitude', 'Unknown')
    return "Unknown"

def get_longitude(laureate):
    if 'birth' in laureate and 'place' in laureate['birth']:
        return laureate['birth']['place'].get('city', {}).get('longitude', 'Unknown')
    return "Unknown"

laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:
        laureate = response.json()['laureates'][0]
        laureates_dict["ID"].append(id)
        for key, func in functions_dict.items():
            if func:
                laureates_dict[key].append(func(laureate))
            else:
                laureates_dict[key].append(id)
     
laureates_df = pd.DataFrame(laureates_dict)

laureates_df



KeyError: 'laureate_ids'

# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [ ]:
# Your code here